# Robust DPatch Attack

## Setup

Below we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [ ]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Dioptra API ports
RESTAPI_PORT = "20080"
MLFLOW_TRACKING_PORT = "25000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the DIOPTRA_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Path to custom task plugins archives
CUSTOM_PLUGINS_EVALUATION_TAR_GZ = Path("custom-plugins-evaluation.tar.gz")

# Path to custom task plugins archives
CUSTOM_RCNN_EVALUATION_TAR_GZ = Path("custom-plugins-rcnn.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = "jtsexton_patch_rcnn"

# Path to dataset
data_path_mnist = "/nfs/data/Mnist"

# Path to saved model
load_model_path = "/nfs/saved-models/stop_sign_training_outputs/output/"

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

In [ ]:
%%bash

# Create the workflows.tar.gz file
make workflows

In [ ]:
restapi_client = utils.DioptraClient()

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

In [ ]:
# Delete the 'evaluation' custom task plugin package
restapi_client.delete_custom_task_plugin(name="evaluation")
restapi_client.delete_custom_task_plugin(name="rcnn")

In [ ]:
%%bash

# Create the workflows.tar.gz file
rm custom-plugins-rcnn.tar.gz
rm custom-plugins-evaluation.tar.gz
make custom-plugins

In [ ]:
response_custom_plugins = restapi_client.get_custom_task_plugin(name="evaluation")
#restapi_client.delete_custom_task_plugin(name="evaluation")
if response_custom_plugins is None or "Not Found" in response_custom_plugins.get("message", []):
    response_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="evaluation",
        custom_plugin_file=CUSTOM_PLUGINS_EVALUATION_TAR_GZ,
    )
response_custom_plugins = restapi_client.get_custom_task_plugin(name="rcnn")
#restapi_client.delete_custom_task_plugin(name="evaluation")
if response_custom_plugins is None or "Not Found" in response_custom_plugins.get("message", []):
    response_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="rcnn",
        custom_plugin_file=CUSTOM_RCNN_EVALUATION_TAR_GZ,
    )

In [ ]:
response_attack = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="attack",
    entry_point_kwargs=" ".join([
        f"-P model_loc={load_model_path}",
        "-P image_size=400,300,3",
        "-P patch_shape=50,50,3",
        "-P patch_location=2,2",
        "-P crop_range=0,0",
        "-P brightness_range=0.1,1.9",
        "-P rotation_weights=1,0,0,0",
        "-P sample_size=1",
        "-P learning_rate=0.1",
        "-P lr_decay_size=0.95",
        "-P lr_decay_schedule=5000",
        "-P momentum=0.9",
        "-P max_iter=30",
        "-P batch_size=20",
        "-P detection_score_cutoff=0.5"
   ]),
    queue="pytorch_gpu",
    timeout="48hr",
)

pprint.pprint(response_attack)